## Projecting emissions with the kaya identity

- Using data from Our World in Data for population and GDP for 1970-2023
- Use 1970 values from Our World in Data and apply historical improvement rates for CO2 intensity of energy and Energy intensity of economy to project to 2023 (from class reading)
- Apply Kaya Identity to estimate historical CO2 emissions
- Based on class reading, consider how emissions are different in different wealth classes 

** Students must enter values in section 2 and in section 5 **

For EESC4020 Humans and the Carbon Cycle, Columbia University, Fall 2024 \
    - Developed by G.A. McKinley. Oct 2024

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget 
# ^ allows zooming in to the plots!

import cmocean as cm    

import gcsfs
fs = gcsfs.GCSFileSystem()

%run plotting_functions.ipynb

## 0. Set Constants

In [ ]:
# Define the constants
start_year = 1970
end_year = 2023

## 1. Load Population and GDP from 1970-2023

https://ourworldindata.org/population-growth#explore-data-on-population-growth \
https://ourworldindata.org/grapher/national-gdp-constant-usd-wb?tab=chart&country=~OWID_WRL


In [ ]:
population = pd.read_csv('WorldPopulation19702023.csv')

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Population')
plt.title('Population')
plt.plot(population["year"],population["world_population"], color='c')
plt.grid(True)
plt.show()

In [ ]:
column_names = ["Year", "World_GDP"]
GDP = pd.read_csv("GDP19702023.csv", names=column_names)

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('GDP (2015 USD)')
plt.title('World GDP')
plt.plot(GDP["Year"],GDP["World_GDP"], color='g')
plt.grid(True)
plt.show()

## 2. Using the historical growth rate of CO$_2$ intensity of energy and the 1970 initial value, estimate the timeseries 1970-2023

In [ ]:
# 1970 CO2 Intensity of Energy (Our World in Data) =  0.26 kgCO$_2$/kWh
CO2Energy1970 = 0.26

In [ ]:
# Rate of historical improvement in CO2 Intensity of Energy (1971-2010, from reading/lecture)
# *** If the data you enter is given in percentage, it needs to be divided by 100 *** 
# *** An "improvement" here means a negative number  *** 
rCO2Energy =    ## Student enters value here
# *** If the data you enter is given in percentage, it needs to be divided by 100 *** 
# *** An "improvement" here means a negative number  *** 

In [ ]:
# Use the initial value and the 
# Initialize the CO2Energy dictionary with 1970 value
CO2Energy = {1970: CO2Energy1970}  
# Calculate CO2Energy for each year
for year in range(start_year + 1, end_year + 1):
    CO2Energy[year] = CO2Energy[year - 1] *(1 + rCO2Energy) 

In [ ]:
years = list(CO2Energy.keys())
CO2Energy19702023 = np.array(list(CO2Energy.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('CO$_2$ energy intensity (kgCO$_2$/kWh)')
plt.title('CO$_2$ energy intensity')
plt.plot(years, CO2Energy19702023, color='b', label='CO₂ Energy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 1970 Energy Intensity of the Economy = 2.9 kWh/$GDP   (primary energy)
EnergyInt1970 = 2.9

In [ ]:
# Rate of historical improvement in Energy Intensity of the Economy (1971-2010, from reading/lecture)
# *** If the data you enter is given in percentage, it needs to be divided by 100 *** 
# *** An "improvement" here means a negative number  *** 
rEnergyInt =    ## Student enters value here
# *** If the data you enter is given in percentage, it needs to be divided by 100 *** 
# *** An "improvement" here means a negative number  *** 

In [ ]:
# Use the initial value and the 
# Initialize the Energy Intensity dictionary with 1970 value
EnergyInt = {1970: EnergyInt1970}  
# Calculate Energy Intensity for each year
for year in range(start_year + 1, end_year + 1):
    EnergyInt[year] = EnergyInt[year - 1] * (1+ rEnergyInt)

In [ ]:
years = list(EnergyInt.keys())
EnergyInt19702023 = np.array(list(EnergyInt.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Energy intensity of Economy (kWh/GDP)')
plt.title('Energy intensity of Economy')
plt.plot(years, EnergyInt19702023, color='r', label='Energy intensity')
plt.legend()
plt.grid(True)
plt.show()

## 3. Use Kaya Identity to estimate emissions 1970-2023

Combine the values above to estimate historical CO$_2$ Emissions

Actual 2023 emissions were 35.3 GtCO2/yr (Global Carbon Budget 2023, Friedlingstein et al. 2023). \
Your numbers will not be exact, but you should be in the ballpark (+- 5 GtCO2/yr)

In [ ]:
GDPdata = np.array(GDP["World_GDP"])
population_data = np.array(population["world_population"])

In [ ]:
CO2emission = CO2Energy19702023*EnergyInt19702023*(GDPdata/population_data)*population_data

In [ ]:
CO2emissionPgCyr = CO2emission*1000/1e15 # in PgC/yr

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('CO$_2$ Emission (PgC/yr)')
plt.title('Historical CO$_2$ Emissions 1970-2023')
plt.plot(years, CO2emissionPgCyr, color='k', label='CO$_2$ Emission')
plt.legend()
plt.grid(True)
plt.show()

## 4. Estimate per capita Emissions

In [ ]:
CO2emission_percapita = CO2Energy19702023*EnergyInt19702023*(GDPdata/population_data)/1000

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('CO$_2$ Emission (ton CO$_2$/person/yr)')
plt.title('Historical CO$_2$ Emissions per capita 1970-2023')
plt.plot(years, CO2emission_percapita, color='k', label='CO$_2$ Emission per capita')
plt.legend()
plt.grid(True)
plt.show()

## 5. Breakdown the per capita emissions by wealth groups: top 10%, middle 40%, bottom 50%

Based on the reading and lecture, use reasonable values to distinguish the per capita emissions of these groups. 

It is not the case that these distinctions have been constant over 1970-2023, but for simplicity, we apply them as constants

The sum of your "split" should be roughly (not perfectly is OK!) equal to the world per capita (compare the bold and dash solid lines in the graph to ensure this)

In [ ]:
# Top 10% Factor
factor10 =  # Student Enters Value Here
# Middle 40% Factor
factor40 =  # Student Enters Value Here
# Bottom 50% Factor
factor50 =   # Student Enters Value Here

In [ ]:
CO2emission_percapita_split = .1*CO2emission_percapita*factor10  + .4*CO2emission_percapita*factor40 + .5*CO2emission_percapita*factor50

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('CO$_2$ Emission (ton CO$_2$/person/yr)')
plt.title('Historical CO$_2$ Emissions per capita 1970-2023')
plt.plot(years, CO2emission_percapita, color='k', label='CO$_2$ Emission per capita')
plt.plot(years, CO2emission_percapita_split, color='k', linestyle = '--', label='CO$_2$ Emission per capita sum of your split')
plt.plot(years, CO2emission_percapita*factor10, color='r',linestyle = '--', label=('CO$_2$ Emission per capita, top 10%; x' + str(round(factor10,1))))
plt.plot(years, CO2emission_percapita*factor40, color='g', linestyle = '--',label=('CO$_2$ Emission per capita, middle 40%; x' + str(round(factor40,1))))
plt.plot(years, CO2emission_percapita*factor50, color='b',linestyle = '--', label=('CO$_2$ Emission per capita, bottom 50%; x' + str(round(factor50,1))))
plt.legend()
plt.grid(True)
plt.show()